In [ ]:
import import_ipynb

In [ ]:
from Solver import ThomasBoundaryCondition
from Conversion import *
import DistanceMap as dis
import SpatialDiscretization as grid
import numpy as np
import pandas as pd

In [ ]:
## Constant parameters
g = 9.8065 # Unit m/s2                
waterDensity = 1000.0 # Unit kg/m3    
area = pow(dis.resolution, 2)
maxNrIterations = 10000
tolerance = 0.0001

In [ ]:
hor = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))   
vol = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))    
a = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))       
b = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))      
c = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))      
d = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))      
psi = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))
dpsi = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))
theta = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))  
oldTheta = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))  
C = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))
k = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))    
u = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))       
du = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))     
f = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))    
H = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))       
H0 = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max()))) 
z = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))  
dz = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))  
zCenter = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num)), int(dis.dem.max())))  
SoilLayer_initial = np.zeros((len(dis.Cat_indi), int(max(dis.Cat_grid_num))))  

In [ ]:
# Psi = pd.read_csv('Data/SSPH_forcing_VtiltCatchment_SCL.csv', header=0, parse_dates=[0], index_col=0)

In [ ]:
def initialization(Cat, Cat_grid, Soil, Se_init, soildepth, SoilGrid_height): 

    SoilLayer_initial[Cat][Cat_grid] = round(soildepth / SoilGrid_height)

    ## z: the location of nodes
    z = grid.linear(Cat, Cat_grid, int(SoilLayer_initial[Cat][Cat_grid]), soildepth)
    
    vol[Cat][Cat_grid][0] = 0.0
    for i in range(int(SoilLayer_initial[Cat][Cat_grid])+1): 
        dz[Cat][Cat_grid][i] = z[Cat][Cat_grid][i+1]-z[Cat][Cat_grid][i] 
        if (i > 0): vol[Cat][Cat_grid][i] = area * dz[Cat][Cat_grid][i]

    for i in range(int(SoilLayer_initial[Cat][Cat_grid])+2): 
        zCenter[Cat][Cat_grid][i] = z[Cat][Cat_grid][i] + dz[Cat][Cat_grid][i] * 0.5
        
    for i in range(int(SoilLayer_initial[Cat][Cat_grid])+1):
        dz[Cat][Cat_grid][i] = zCenter[Cat][Cat_grid][i+1] - zCenter[Cat][Cat_grid][i]

    # Initialization
    for i in range(1, int(SoilLayer_initial[Cat][Cat_grid])+2):
        hor[Cat][Cat_grid][i] = getHorizonIndex(Soil, zCenter[Cat][Cat_grid][i])
        theta[Cat][Cat_grid][i] = thetaFromSe(Soil[int(hor[Cat][Cat_grid][i])], Se_init) # determination of theta of each soil layer
        ## psi unit = kPa
        psi[Cat][Cat_grid][i] = waterPotential(Soil[int(hor[Cat][Cat_grid][i])], theta[Cat][Cat_grid][i]) # psi is matric potential (matric suction head) and also can be assumed as total soil water potential in head unit
        
        if i <= int(SoilLayer_initial[Cat][Cat_grid]):
            H[Cat][Cat_grid][i] = psi[Cat][Cat_grid][i] - (zCenter[Cat][Cat_grid][i] * g) # H is total head (unit= J/kg)
        
        elif i == int(SoilLayer_initial[Cat][Cat_grid])+1:
            ## Constant pressure head - Lower (GWT: zero matric potential) 
            Lower_BC = -2.6 ## Unit: kPa
            psi[Cat][Cat_grid][i] = Lower_BC * 1000 / waterDensity ## Unit: kPa
            theta[Cat][Cat_grid][i] = thetaFromPsi(Soil[int(hor[Cat][Cat_grid][i])], psi[Cat][Cat_grid][i])
            ## zCenter[i] * g = m2/s2 = J/kg
            H[Cat][Cat_grid][i] = psi[Cat][Cat_grid][i] - (zCenter[Cat][Cat_grid][i] * g)
            
    return SoilLayer_initial[Cat][Cat_grid]

In [ ]:
def Updating_depth(Cat, Cat_grid, Soil, Time_varying_depth, SoilLayer_num):
                
    z = grid.linear(Cat, Cat_grid, int(SoilLayer_num), Time_varying_depth)

    vol[Cat][Cat_grid][0] = 0.0
    for i in range(int(SoilLayer_num)+1): 
        dz[Cat][Cat_grid][i] = z[Cat][Cat_grid][i+1] - z[Cat][Cat_grid][i] # uniform spacing
        if (i > 0): vol[Cat][Cat_grid][i] = area * dz[Cat][Cat_grid][i]

    for i in range(int(SoilLayer_num)+2): 
        zCenter[Cat][Cat_grid][i] = z[Cat][Cat_grid][i] + dz[Cat][Cat_grid][i] * 0.5
    
    for i in range(int(SoilLayer_num)+1):
        dz[Cat][Cat_grid][i] = zCenter[Cat][Cat_grid][i+1] - zCenter[Cat][Cat_grid][i]

In [ ]:
def infiltration_by_FVM(Cat, Cat_grid, Soil, dt, meanType, FrameNumber, SoilLayer_num): 
    
    nt_soil = 1
    maxTimestep = dt ## dt: 3600 s
    dt_soil = maxTimestep / nt_soil
    
#     # Time-dependent SSPH upper soil column boundary condition
#     psi[Cat][Cat_grid][0] = Psi['Psi_init'][int(FrameNumber)+1] ## Unit: kPa 

    ## Constant SSPH upper soil column boundary condition 
    Upper_BC = -100.0 ## Unit: kPa
    psi[Cat][Cat_grid][0] = Upper_BC * 1000 / waterDensity ## Unit: m2/s2 (kPa)
    
    theta[Cat][Cat_grid][0] = thetaFromPsi(Soil[0], psi[Cat][Cat_grid][0])
    theta[Cat][Cat_grid][1] = theta[Cat][Cat_grid][0]
    psi[Cat][Cat_grid][1] = psi[Cat][Cat_grid][0]

    ## Initial Condition (I.C)
    sum0 = 0
    for i in range(1, int(SoilLayer_num)+2):
        if i <= int(SoilLayer_num):
            H0[Cat][Cat_grid][i] = psi[Cat][Cat_grid][i] - zCenter[Cat][Cat_grid][i] * g
        elif i == int(SoilLayer_num)+1:
            H0[Cat][Cat_grid][i] = waterPotential(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
                                                  thetaFromSe(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])], 1.0))\
                                                - zCenter[Cat][Cat_grid][i] * g
        H[Cat][Cat_grid][i] = H0[Cat][Cat_grid][i]
        if i <= int(SoilLayer_num):
            sum0 += waterDensity * vol[Cat][Cat_grid][i] * theta[Cat][Cat_grid][i] 
    
    massBalance = sum0    
    
    nrIterations = 0
    while ((massBalance > tolerance) and (nrIterations < maxNrIterations)):

        for i in range(1, int(SoilLayer_num)+1):
            k[Cat][Cat_grid][i] = hydraulicConductivityFromTheta(Soil[int(hor[Cat][Cat_grid][i])], theta[Cat][Cat_grid][i])
            capacity = dTheta_dH(Soil[int(hor[Cat][Cat_grid][i])], H0[Cat][Cat_grid][i], H[Cat][Cat_grid][i], zCenter[Cat][Cat_grid][i])
            C[Cat][Cat_grid][i] = (waterDensity * vol[Cat][Cat_grid][i] * capacity) / (dt_soil)
            
            if i <= int(SoilLayer_num)-1:
                f[Cat][Cat_grid][i] = area * meanK(meanType, k[Cat][Cat_grid][i], k[Cat][Cat_grid][i+1]) / dz[Cat][Cat_grid][i]
        
        f[Cat][Cat_grid][0] = 0.0
        ## meank unit: kg*s/m3 kg per unit volume per second
        f[Cat][Cat_grid][int(SoilLayer_num)] = area * meanK(meanType, k[Cat][Cat_grid][int(SoilLayer_num)],\
                                                       hydraulicConductivityFromTheta(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
                                                        thetaFromPsi(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
                                                        psi[Cat][Cat_grid][int(SoilLayer_num)+1]))) / dz[Cat][Cat_grid][int(SoilLayer_num)]            
        
       
        for i in range(1, int(SoilLayer_num)+1):    
            
            a[Cat][Cat_grid][i] = -f[Cat][Cat_grid][i-1]
            if (i == 1):
                b[Cat][Cat_grid][i] = 1
                c[Cat][Cat_grid][i] = 0
                d[Cat][Cat_grid][i] = H0[Cat][Cat_grid][i] # point where Upper B.C is applied 
            elif (i < int(SoilLayer_num)):
                b[Cat][Cat_grid][i] = C[Cat][Cat_grid][i] + f[Cat][Cat_grid][i-1] + f[Cat][Cat_grid][i]
                c[Cat][Cat_grid][i] = -f[Cat][Cat_grid][i]
                d[Cat][Cat_grid][i] = C[Cat][Cat_grid][i] * H0[Cat][Cat_grid][i]
            elif (i == int(SoilLayer_num)):
                b[Cat][Cat_grid][i] = C[Cat][Cat_grid][i] + f[Cat][Cat_grid][i-1]
                c[Cat][Cat_grid][i] = 0
                d[Cat][Cat_grid][i] = C[Cat][Cat_grid][i] * H0[Cat][Cat_grid][i] - f[Cat][Cat_grid][i] *\
                (H[Cat][Cat_grid][i] - H[Cat][Cat_grid][i+1])
        
        ThomasBoundaryCondition(a[Cat, Cat_grid, :], b[Cat, Cat_grid, :], c[Cat, Cat_grid, :], d[Cat, Cat_grid, :],\
                                H[Cat, Cat_grid, :], 1, int(SoilLayer_num)) 
        
        newSum = 0
        for i in range(1, int(SoilLayer_num)+1):
            psi[Cat][Cat_grid][i] = H[Cat][Cat_grid][i] + zCenter[Cat][Cat_grid][i] * g #H is renewed hydraulic head for each time step
            theta[Cat][Cat_grid][i] = thetaFromPsi(Soil[int(hor[Cat][Cat_grid][i])], psi[Cat][Cat_grid][i]) 
            newSum += waterDensity * vol[Cat][Cat_grid][i] * theta[Cat][Cat_grid][i] # newSum is the mass of water in the grid-cell
        
        ## newSum = kg, sum0 = kg, f(H1-H2)dt = kg
        massBalance = abs(newSum - (sum0 + (f[Cat][Cat_grid][1] * (H[Cat][Cat_grid][1] - H[Cat][Cat_grid][2]) * (dt_soil))\
                                    - (f[Cat][Cat_grid][int(SoilLayer_num)] * (H[Cat][Cat_grid][int(SoilLayer_num)]\
                                                                               - H[Cat][Cat_grid][int(SoilLayer_num)+1]) * (dt_soil))))
        nrIterations += 1
       
    if (massBalance < tolerance): 
        ## flux unit = m3
        Upper_flux = f[Cat][Cat_grid][1] * ((H[Cat][Cat_grid][1] - H[Cat][Cat_grid][2]) * (1 / waterDensity)) * (dt_soil)
        Lower_flux = f[Cat][Cat_grid][int(SoilLayer_num)] * ((H[Cat][Cat_grid][int(SoilLayer_num)]\
                                                                  - H[Cat][Cat_grid][int(SoilLayer_num)+1]) * (1 / waterDensity)) * (dt_soil)
#         if (Cat==0 and Cat_grid==9):
#             print('Time=', format(FrameNumber, '.1f'), 'Cat=', format(Cat, '.1f'), 'Cat_grid=', format(Cat_grid, '.1f'),
#                   'meanK=', format(meanK(meanType, k[Cat][Cat_grid][int(SoilLayer_num)],\
#                                                        hydraulicConductivityFromTheta(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
#                                                         thetaFromPsi(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
#                                                         psi[Cat][Cat_grid][int(SoilLayer_num)+1]))), '.10f'),
#                   'dz[n]=', format(dz[Cat][Cat_grid][int(SoilLayer_num)], '.3f'),
#                   'f[n]=', format(f[Cat][Cat_grid][int(SoilLayer_num)], '.3f'),
#                   'H[n]=', format(H[Cat][Cat_grid][int(SoilLayer_num)], '.3f'),
#                   'H[n+1]=', format(H[Cat][Cat_grid][int(SoilLayer_num)+1], '.3f'))
        
        return True, nrIterations, Upper_flux, Lower_flux
        
    else:
        print("massBal=", format(massBalance, '.3f'), 'k[n]=',\
              format(k[Cat][Cat_grid][int(SoilLayer_num)], '.10f'), 'k[n+1]=',\
              format(hydraulicConductivityFromTheta(Soil[hor[Cat][Cat_grid][int(SoilLayer_num)+1]],\
                                                    thetaFromPsi(Soil[hor[Cat][Cat_grid][int(SoilLayer_num)+1]],\
                                                                 psi[Cat][Cat_grid][int(SoilLayer_num)+1])), '.10f'), 'H[1]=', format(H[Cat][Cat_grid][1], '.3f'), 'H[2]=', format(H[Cat][Cat_grid][2], '.3f'), 'H[n]=', format(H[Cat][Cat_grid][int(SoilLayer_num)], '.3f'), 'H[n+1]=', format(H[Cat][Cat_grid][int(SoilLayer_num)+1], '.3f'),\
              'dz[n]=', format(dz[int(SoilLayer_num)], '.3f'), 'meanK[n]=', format(meanK(meanType, k[Cat][Cat_grid][int(SoilLayer_num)], hydraulicConductivityFromTheta(Soil[hor[Cat][Cat_grid][int(SoilLayer_num)+1]],\
                                                                              thetaFromPsi(Soil[int(hor[Cat][Cat_grid][int(SoilLayer_num)+1])],\
                                                                                           psi[Cat][Cat_grid][int(SoilLayer_num)+1]))), '.10f'))
        return False, nrIterations, 0, dt_soil     